In [0]:
#Importing the reqired modules
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN
from keras.layers import Dense

In [2]:
#Mounting drive folder to colab for access of features and labels data
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
cd '/content/drive/My Drive/Btech - Mechanical Engineering/Sem 4/Applied Machine Learning/Drowsiness Detection Project/Drowsiness/videos/data'

/content/drive/.shortcut-targets-by-id/1CZPaoJcaUKraM0cNUJ1AiIOMZHGg3jSI/Drowsiness/videos/data


# Loading Data

In [0]:
features_csv=["./features_"+str(x)+".csv" for x in range(1,31)]
labels_csv=["./labels_"+str(x)+".csv" for x in range(1,31)]

In [0]:
features=pd.read_csv(features_csv[1],header=None)
labels=pd.read_csv(labels_csv[0],header=None)

In [6]:
features.shape

(470, 4)

In [0]:
conc=pd.concat([features,labels],axis=1,ignore_index=True)
conc.replace(10,1,inplace=True)

In [8]:
conc_0=conc.loc[conc[4]==0]
conc_1=conc.loc[conc[4]==1]
print(conc_0)
print(conc_1)

          0         1        2         3    4
0  0.252851  0.430504  0.38903  1.702598  0.0
          0         1        2         3    4
1  0.173133  0.426659  0.30192  2.464341  1.0


In [9]:
features=features[:(features.shape[0]//5)*5]
features.shape

(470, 4)

In [0]:
features=[]
labels=[]

In [11]:
for a in range(len(features_csv)):
  try:
    feat=pd.read_csv(features_csv[a],header=None)
    lab=pd.read_csv(labels_csv[a],header=None)
    lab.replace(10,1,inplace=True)
    feat_0=feat.loc[lab[0]==0]
    feat_1=feat.loc[lab[0]==1]
    lab_0=lab.loc[lab[0]==0]
    lab_1=lab.loc[lab[0]==1]

    feat_0=feat_0[:(feat_0.shape[0]//5)*5]
    feat_1=feat_1[:(feat_1.shape[0]//5)*5]
    lab_0=lab_0[:(lab_0.shape[0]//5)*5]
    lab_1=lab_1[:(lab_1.shape[0]//5)*5]
    features.append(feat_0)
    features.append(feat_1)
    labels.append(lab_0)
    labels.append(lab_1)
  except:
    print("Empty File")

Empty File


In [0]:
features_df=features[0]
for a in features:
  features_df=features_df.append(a)

In [93]:
features_df.shape

(8680, 4)

In [0]:
(features_df[3]==np.inf).value_counts()
features_df.replace(np.inf,np.nan,inplace=True)
features_df.fillna(features_df.mean(), inplace=True)

In [95]:
features_df.shape

(8680, 4)

In [0]:
features_df=features_df.values

In [97]:
features_df.shape

(8680, 4)

In [0]:
features_df=features_df.reshape(features_df.shape[0]//5,5,4)

In [99]:
features_df.shape

(1736, 5, 4)

In [0]:
labels_df=labels[0]
for a in labels:
  labels_df=labels_df.append(a)

In [101]:
labels_df=labels_df.values
labels_df.shape

(8680, 1)

In [102]:
labels_df.reshape(-1,1)
labels_df[10]

array([0.])

In [0]:
labels_np=[]
for a in range(0,8680,5):
  labels_np.append(labels_df[a][0])
labels_np=np.array(labels_np)

In [104]:
labels_np=labels_np.reshape(-1,1)
labels_np.shape

(1736, 1)

# Training LSTM model

In [0]:
x_train,x_test,y_train,y_test=train_test_split(features_df,labels_np,test_size=0.2,random_state=30)

In [106]:
model = Sequential()
model.add(LSTM((128), batch_input_shape = (None, 5,4), return_sequences = True))
model.add(LSTM((100), activation = 'softmax', return_sequences = True))
model.add(LSTM((1), activation = 'softmax', return_sequences = False))
model.compile(loss = 'mse',  optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_36 (LSTM)               (None, 5, 128)            68096     
_________________________________________________________________
lstm_37 (LSTM)               (None, 5, 100)            91600     
_________________________________________________________________
lstm_38 (LSTM)               (None, 1)                 408       
Total params: 160,104
Trainable params: 160,104
Non-trainable params: 0
_________________________________________________________________


In [110]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100)

Train on 1388 samples, validate on 348 samples
Epoch 1/100
1388/1388 [==============================] - 1s 591us/step - loss: 0.1101 - accuracy: 0.8545 - val_loss: 0.1645 - val_accuracy: 0.7787
Epoch 2/100
1388/1388 [==============================] - 1s 570us/step - loss: 0.1032 - accuracy: 0.8718 - val_loss: 0.1695 - val_accuracy: 0.7874
Epoch 3/100
1388/1388 [==============================] - 1s 581us/step - loss: 0.1056 - accuracy: 0.8646 - val_loss: 0.1733 - val_accuracy: 0.7701
Epoch 4/100
1388/1388 [==============================] - 1s 569us/step - loss: 0.0990 - accuracy: 0.8775 - val_loss: 0.1681 - val_accuracy: 0.7730
Epoch 5/100
1388/1388 [==============================] - 1s 578us/step - loss: 0.1021 - accuracy: 0.8696 - val_loss: 0.1674 - val_accuracy: 0.7672
Epoch 6/100
1388/1388 [==============================] - 1s 569us/step - loss: 0.1022 - accuracy: 0.8739 - val_loss: 0.1794 - val_accuracy: 0.7471
Epoch 7/100
1388/1388 [==============================] - 1s 592us/step 

In [115]:
y_pred=model.predict(x_test)>0.5
y_pred=y_pred.astype(int)
y_pred

array([[0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
    

In [116]:
#Accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test.reshape(-1),y_pred)

0.7586206896551724

In [118]:
from sklearn.metrics import confusion_matrix
cmx=confusion_matrix(y_pred,y_test)
cmx

array([[136,  36],
       [ 48, 128]])

In [119]:
#Precision
cmx[1][1]/(cmx[1][1]+cmx[1][0])

0.7272727272727273